# Portfolio Analysis Tool - Interactive Demo

This notebook demonstrates the capabilities of our portfolio analysis tool.
It showcases portfolio metrics calculation, efficient frontier analysis, and visualization features.

In [ ]:
# Import required libraries and modules
import sys
import os

# Add the scripts directory to the path
sys.path.append(os.path.join('..', 'scripts'))

from scripts.data_fetcher import fetch_data
from scripts.portfolio_metrics import calculate_portfolio_metrics
from scripts.visualization import plot_allocation, plot_efficient_frontier, plot_optimal_portfolios
from scripts.input_handling import create_sample_portfolio
from scripts.efficient_frontier import (
    calculate_efficient_frontier, 
    suggest_optimal_weights,
    find_max_sharpe_portfolio,
    find_min_volatility_portfolio
)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("All modules imported successfully!")

## 1. Create Sample Portfolio

Let's start by creating a sample portfolio with major tech stocks.

In [ ]:
# Create sample portfolio
portfolio_df = create_sample_portfolio()
print("Sample Portfolio:")
print(portfolio_df)
print(f"\nTotal Weight: {portfolio_df['Weight'].sum():.2f}")

## 2. Fetch Historical Data

Download 5 years of historical data for the portfolio assets.

In [ ]:
# Extract assets and fetch data
assets = portfolio_df['Ticker'].tolist()
weights = portfolio_df['Weight'].tolist()

print("Fetching historical data...")
all_data = fetch_data(assets)

print(f"\nData shape: {all_data.shape}")
print(f"Date range: {all_data.index.min()} to {all_data.index.max()}")
print(f"Assets: {list(all_data.columns)}")

# Display first few rows
print("\nFirst 5 rows of data:")
all_data.head()

## 3. Calculate Portfolio Metrics

Calculate key performance metrics for our portfolio.

In [ ]:
# Ensure weights match available assets
remaining_assets = all_data.columns.tolist()
remaining_weights = [weights[assets.index(asset)] for asset in remaining_assets if asset in assets]
total_weight = sum(remaining_weights)
normalized_weights = [w / total_weight for w in remaining_weights]

# Calculate portfolio metrics
metrics = calculate_portfolio_metrics(all_data, normalized_weights)

print("Portfolio Performance Metrics:")
print("=" * 40)
for key, value in metrics.items():
    if "Return" in key or "Ratio" in key:
        print(f"{key}: {value:.2%}" if "Return" in key else f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value:.2%}")

## 4. Visualize Portfolio Allocation

Create a pie chart showing the portfolio allocation.

In [ ]:
# Plot portfolio allocation
plot_allocation(normalized_weights, remaining_assets)

## 5. Efficient Frontier Analysis

Generate the efficient frontier and analyze portfolio optimization opportunities.

In [ ]:
# Calculate efficient frontier
print("Calculating efficient frontier (this may take a moment...)")
results = calculate_efficient_frontier(all_data, num_portfolios=5000)

print(f"Generated {len(results['returns'])} random portfolios")
print(f"Return range: {min(results['returns']):.2%} to {max(results['returns']):.2%}")
print(f"Risk range: {min(results['risks']):.2%} to {max(results['risks']):.2%}")
print(f"Sharpe ratio range: {min(results['sharpe_ratios']):.4f} to {max(results['sharpe_ratios']):.4f}")

In [ ]:
# Plot efficient frontier with current portfolio highlighted
plot_efficient_frontier(results, metrics)

In [ ]:
# Create Interactive Efficient Frontier with Plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

def create_interactive_efficient_frontier(results, metrics, assets, weights):
    """Create an interactive efficient frontier plot with portfolio details"""
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Interactive Efficient Frontier', 'Portfolio Allocation'),
        specs=[[{"secondary_y": False}, {"type": "pie"}]],
        column_widths=[0.7, 0.3]
    )
    
    # Add efficient frontier scatter plot
    fig.add_trace(
        go.Scatter(
            x=results['risks'],
            y=results['returns'],
            mode='markers',
            marker=dict(
                size=8,
                color=results['sharpe_ratios'],
                colorscale='Viridis',
                colorbar=dict(title="Sharpe Ratio"),
                opacity=0.7
            ),
            text=[f"Return: {r:.2%}<br>Risk: {risk:.2%}<br>Sharpe: {s:.3f}" 
                  for r, risk, s in zip(results['returns'], results['risks'], results['sharpe_ratios'])],
            hovertemplate="%{text}<extra></extra>",
            name="Efficient Frontier"
        ),
        row=1, col=1
    )
    
    # Highlight current portfolio
    fig.add_trace(
        go.Scatter(
            x=[metrics['Risk (Annualized Std Dev)']],
            y=[metrics['Expected Return (CAGR)']],
            mode='markers',
            marker=dict(size=15, color='red', symbol='star'),
            name="Your Portfolio",
            text=f"Your Portfolio<br>Return: {metrics['Expected Return (CAGR)']:.2%}<br>Risk: {metrics['Risk (Annualized Std Dev)']:.2%}",
            hovertemplate="%{text}<extra></extra>"
        ),
        row=1, col=1
    )
    
    # Add optimal portfolios
    max_sharpe = find_max_sharpe_portfolio(results)
    min_vol = find_min_volatility_portfolio(results)
    
    fig.add_trace(
        go.Scatter(
            x=[max_sharpe['risk']],
            y=[max_sharpe['return']],
            mode='markers',
            marker=dict(size=12, color='gold', symbol='diamond'),
            name="Max Sharpe",
            text=f"Max Sharpe<br>Return: {max_sharpe['return']:.2%}<br>Risk: {max_sharpe['risk']:.2%}",
            hovertemplate="%{text}<extra></extra>"
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=[min_vol['risk']],
            y=[min_vol['return']],
            mode='markers',
            marker=dict(size=12, color='lightblue', symbol='diamond'),
            name="Min Vol",
            text=f"Min Vol<br>Return: {min_vol['return']:.2%}<br>Risk: {min_vol['risk']:.2%}",
            hovertemplate="%{text}<extra></extra>"
        ),
        row=1, col=1
    )
    
    # Add pie chart for current portfolio
    fig.add_trace(
        go.Pie(
            labels=assets,
            values=weights,
            name="Portfolio Allocation",
            textinfo='label+percent',
            textposition='inside'
        ),
        row=1, col=2
    )
    
    # Update layout
    fig.update_layout(
        title="Interactive Portfolio Analysis Dashboard",
        height=600,
        showlegend=True,
        hovermode='closest'
    )
    
    # Update axes
    fig.update_xaxes(title_text="Risk (Annualized Std Dev)", row=1, col=1)
    fig.update_yaxes(title_text="Expected Return (CAGR)", row=1, col=1)
    
    return fig

# Create and display interactive plot
interactive_fig = create_interactive_efficient_frontier(results, metrics, remaining_assets, normalized_weights)
interactive_fig.show()

In [ ]:
# Interactive Portfolio Explorer
# Click on points in the above plot to explore different portfolios!

def explore_portfolio_by_index(portfolio_index):
    """Explore a specific portfolio from the efficient frontier by index"""
    if 0 <= portfolio_index < len(results['returns']):
        portfolio_weights = results['weights'][portfolio_index]
        portfolio_return = results['returns'][portfolio_index]
        portfolio_risk = results['risks'][portfolio_index]
        portfolio_sharpe = results['sharpe_ratios'][portfolio_index]
        
        print(f"🎯 Portfolio #{portfolio_index} Analysis")
        print("=" * 50)
        print(f"📈 Expected Return: {portfolio_return:.2%}")
        print(f"⚡ Risk (Std Dev): {portfolio_risk:.2%}")
        print(f"📊 Sharpe Ratio: {portfolio_sharpe:.4f}")
        print("\n💰 Asset Allocation:")
        print("-" * 30)
        
        # Create allocation DataFrame
        allocation_df = pd.DataFrame({
            'Asset': remaining_assets,
            'Weight': [f"{w:.1%}" for w in portfolio_weights],
            'Weight_Decimal': portfolio_weights
        })
        
        # Sort by weight
        allocation_df = allocation_df.sort_values('Weight_Decimal', ascending=False)
        
        for _, row in allocation_df.iterrows():
            print(f"{row['Asset']}: {row['Weight']}")
        
        # Create pie chart for this portfolio
        fig_pie = px.pie(
            values=portfolio_weights, 
            names=remaining_assets,
            title=f"Portfolio #{portfolio_index} Allocation (Sharpe: {portfolio_sharpe:.3f})",
            color_discrete_sequence=px.colors.qualitative.Set3
        )
        fig_pie.update_traces(textposition='inside', textinfo='percent+label')
        fig_pie.show()
        
        return allocation_df
    else:
        print(f"❌ Invalid portfolio index. Choose between 0 and {len(results['returns'])-1}")
        return None

# Example: Explore some interesting portfolios
print("🔍 Let's explore some specific portfolios from the efficient frontier:")
print("\n1️⃣ Maximum Sharpe Ratio Portfolio:")
max_sharpe_idx = np.argmax(results['sharpe_ratios'])
explore_portfolio_by_index(max_sharpe_idx)

print("\n2️⃣ Minimum Volatility Portfolio:")
min_vol_idx = np.argmin(results['risks'])
explore_portfolio_by_index(min_vol_idx)

print("\n3️⃣ Try exploring other portfolios by calling:")
print("explore_portfolio_by_index(portfolio_index)")
print(f"Available indices: 0 to {len(results['returns'])-1}")

## 6. Find Optimal Portfolios

Identify optimal portfolios based on different criteria.

In [ ]:
# Find maximum Sharpe ratio portfolio
max_sharpe = find_max_sharpe_portfolio(results)

print("Maximum Sharpe Ratio Portfolio:")
print("=" * 40)
print(f"Expected Return: {max_sharpe['return']:.2%}")
print(f"Risk (Std Dev): {max_sharpe['risk']:.2%}")
print(f"Sharpe Ratio: {max_sharpe['sharpe_ratio']:.4f}")
print("\nOptimal Weights:")
for i, asset in enumerate(remaining_assets):
    print(f"{asset}: {max_sharpe['weights'][i]:.1%}")

In [ ]:
# Find minimum volatility portfolio
min_vol = find_min_volatility_portfolio(results)

print("Minimum Volatility Portfolio:")
print("=" * 40)
print(f"Expected Return: {min_vol['return']:.2%}")
print(f"Risk (Std Dev): {min_vol['risk']:.2%}")
print(f"Sharpe Ratio: {min_vol['sharpe_ratio']:.4f}")
print("\nOptimal Weights:")
for i, asset in enumerate(remaining_assets):
    print(f"{asset}: {min_vol['weights'][i]:.1%}")

In [ ]:
# Plot optimal portfolios on the efficient frontier
plot_optimal_portfolios(results)

## 7. Portfolio Comparison

Compare your current portfolio with the optimal portfolios.

In [ ]:
# Create comparison table
comparison_data = {
    'Portfolio': ['Current Portfolio', 'Max Sharpe Ratio', 'Min Volatility'],
    'Expected Return': [
        f"{metrics['Expected Return (CAGR)']:.2%}",
        f"{max_sharpe['return']:.2%}",
        f"{min_vol['return']:.2%}"
    ],
    'Risk (Std Dev)': [
        f"{metrics['Risk (Annualized Std Dev)']:.2%}",
        f"{max_sharpe['risk']:.2%}",
        f"{min_vol['risk']:.2%}"
    ],
    'Sharpe Ratio': [
        f"{metrics['Sharpe Ratio']:.4f}",
        f"{max_sharpe['sharpe_ratio']:.4f}",
        f"{min_vol['sharpe_ratio']:.4f}"
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("Portfolio Comparison:")
print("=" * 60)
print(comparison_df.to_string(index=False))

## 8. Weight Comparison

Compare asset weights across different portfolio strategies.

In [ ]:
# Create weight comparison
weight_comparison = pd.DataFrame({
    'Asset': remaining_assets,
    'Current Weights': [f"{w:.1%}" for w in normalized_weights],
    'Max Sharpe Weights': [f"{w:.1%}" for w in max_sharpe['weights']],
    'Min Vol Weights': [f"{w:.1%}" for w in min_vol['weights']]
})

print("Weight Allocation Comparison:")
print("=" * 50)
print(weight_comparison.to_string(index=False))

## Conclusion

This notebook demonstrated the key features of our portfolio analysis tool:

1. **Data Fetching**: Automatically downloaded historical data from Yahoo Finance
2. **Portfolio Metrics**: Calculated CAGR, risk, and Sharpe ratio
3. **Visualization**: Created allocation charts and efficient frontier plots
4. **Optimization**: Found portfolios with maximum Sharpe ratio and minimum volatility
5. **Comparison**: Analyzed how current portfolio compares to optimal allocations

The tool provides valuable insights for portfolio optimization and risk management decisions.